# Ho and Lee

In [28]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime, date, time, timedelta

In [29]:
# input variables
r = 0.0541     # interest rate at time t
s = 5       # Maturity
t = 0       # now
T = 0       # expiry
deltat = .1 # deltat for derivative
vol = .23   # volatility
optionType = "C"

In [30]:
# ZCB pricing using Ho and Lee
def ZeroCouponBondPricing(r, s, T, deltat, vol):
    """
    # relate future bond prices at time T >= t to the current term structure and the level of the short rate at time T
    Inputs
    r =      # interest rate at time t
    s =      # Maturity
    T =      # expiry
    deltat = # deltat for derivative
    vol =    # volatility

    Output
    # Price of bond at time T with a maturity of s
    """
    # market prices of T year and s year discount bonds
    pts = np.exp(-r*s)
    ptT = np.exp(-r*T)
    
    B = s - T # B from closed form solution formula
    slope = (np.log(np.exp(-r*(T+deltat))) - np.log(np.exp(-r*(T-deltat))))/(2*deltat) # approximate slope
    lnA = np.log(pts/ptT) - B*(slope) - .5*(vol**2)*(B**2) # lnA(T,s) 
    
    pTs = (np.exp(lnA))*np.exp(-B*r) # Output

    return(pTs)

ZeroCouponBondPricing(r, s, T, deltat, vol)

0.3938638451865482

In [22]:
# european options on ZCB pricing using Ho and Lee
def optionsPricing(r, s, t, T, vol, optionType):
    """
    # Pricing of European call and Put Options on Pure Discount Bonds
    Inputs
    r =          # interest rate at time t
    s =          # Maturity
    t =          # now
    T =          # expiry
    vol =        # volatility
    OptionType = # call or put
    
    Output
    # Price of European Call or Put on Pure Discount Bonds
    """
    
    # Compute constants
    pts = np.exp(-r*s) # price of ZCB with a maturity at s at time t
    ptT = np.exp(-r*T) # price of ZCB with a maturity of T at time t
    K = pts/ptT
    sigmap = vol*(s-T)*np.sqrt(T-t)
    d1 = (np.log(pts/(K*ptT)) / sigmap) + (sigmap/2)
    d2 = d1 - sigmap

    if optionType == "C":
        optionPrice = (pts*norm.cdf(d1)) - (K*ptT*norm.cdf(d2))
    elif optionType == "P":
        optionPrice = (K*ptT*norm.cdf(-d2)) - (pts*norm.cdf(-d1))
    else:
        print("This function only prices European calls and puts")
              
    return(optionPrice)

optionsPricing(r, s, t, T, vol, optionType)

0.012427033940625165

In [23]:
# Read in caplet data
capletData = pd.read_csv("CapletDataFor21Jan1995.csv")

# Additional Caplet Information
t = '21-Jan-95'
capRate = 0.07
Principal = 1
faceValueBond = 1.0175

# convert dates to do time calculations
capletData.iloc[:, 0] = pd.to_datetime(capletData.iloc[:, 0], format='%d-%b-%y')







# Print the DataFrame to check the result
print(capletData)


   Caplet Maturity  Cap. Vol
0       1995-03-21     15.25
1       1995-06-21     17.25
2       1995-09-21     17.25
3       1995-12-21     17.50
4       1996-03-21     18.00
5       1996-06-21     18.00
6       1996-09-21     18.00
7       1996-12-21     18.00
8       1997-03-21     17.75
9       1997-06-21     17.50
10      1997-09-21     17.50
11      1997-12-21     17.50
12      1998-03-21     17.25
13      1998-06-21     17.25


/tmp/ipykernel_92/723488182.py:11: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  capletData.iloc[:, 0] = pd.to_datetime(capletData.iloc[:, 0], format='%d-%b-%y')


In [24]:
from datetime import datetime

sigma = 0.1525
tstring = '1995-01-21'
Tstring = '1995-03-21'

# Use "%Y-%m-%d" for a four-digit year
t = datetime.strptime(tstring, "%Y-%m-%d")
T = datetime.strptime(Tstring, "%Y-%m-%d")

tk = (T - t)
print(tk)


59 days, 0:00:00


In [27]:
from datetime import datetime
# calculating inputs

# I AM NOT GETTING THE RIGHT NUMBERS BUT I THINK THAT IS BECAUSE THEY ARE ROUNDING
# IN THE BOOK. IDK IF I SHOULD ROUND OR NOT

# sigma
# r
# t
# T
# s
# P(0,T)
# P(0,s)
# R(0,T)

sigma = 0.1525
r = .07   
tString = '1995-01-21'
TString = '1995-03-21'
sString = '1995-06-21'

# convert dates to datetimes for calculations
t = datetime.strptime(tString, "%Y-%m-%d")
T = datetime.strptime(TString, "%Y-%m-%d")
s = datetime.strptime(sString, "%Y-%m-%d")

# calculate tk
T = T - t
T = T.days / 365
T = 0.16 # need to delete

# calculate tkPlus1
s = s - t
s = s.days / 365
s = .41 # need to delete

# calculate P(0,T), P(0,s), Spot rate
# NEED TO FIGURE OUT CORRECT BOND PRICING
Price_ZCB_T = 0.9898 #np.exp(-r*T)
Price_ZCB_s = 0.9733 #np.exp(-r*s)
spotRate = -(1/(T))*np.log(Price_ZCB_T)

print(spotRate)

# underlying variable in black model applied to a cplaet is the forward rate applicable to the caplet
forwardRate_t_T_s = 0.0671 # (-1/(s-T))*np.log(Price_ZCB_s/Price_ZCB_T)

# apply blacks formula with K = 0.07 (K = capRate)
# calculate d1, d2
d1 = -0.6513 # (np.log(forwardRate_t_T_s/capRate) + ((sigma**2)/2)*T) / (sigma*np.sqrt(T))
d2 = -0.7126 # d1 - (sigma*np.sqrt(T))

capletValue = Price_ZCB_T * ((forwardRate_t_T_s*norm.cdf(d1)) - capRate*norm.cdf(d2)) * (s-T)*Principal

print(capletValue)

0.06407735290219596
0.0001509561677766183
